In [68]:
import json
import os
import pandas as pd
import numpy as np
from eth_account import account
from web3 import Web3
from src import s3_services
from src import sql_manager
from datetime import datetime
from importlib import reload

In [72]:
sql_manager = reload(sql_manager)


In [2]:
infura_url = 'https://mainnet.infura.io/v3/69765a3368f44f5c8cc3691d2361e496'
web3 = Web3(Web3.HTTPProvider(infura_url))

In [3]:
if web3.isConnected():
    print('The connection with Ethereum network is successfull')
else:
    print('Network connection error')

The connection with Ethereum network is successfull


In [4]:
#latestBlock
block = web3.eth.getBlock('latest', full_transactions=True)
number = block.number
print(f'Searching block {number}')

Searching block 13438609


In [5]:
block_dict = {
    'number': [block.number],
    'timestamp': [block.timestamp],
    'difficulty': [block.difficulty],
    'baseFeePerGas': [block.baseFeePerGas],
    'gasLimit':[block.gasLimit],
    'gasUsed': [block.gasUsed],
    'hash': [web3.toHex(block.hash)],
    'miner': [block.miner],
    'size': [block.size],
    'nonce': [web3.toHex(block.nonce)]
}

In [6]:
# DataFrame conversion
block_data = pd.DataFrame(block_dict)

trx_data = pd.DataFrame.from_dict(block.transactions)

In [39]:
block_data.dtypes

number            int64
timestamp         int64
difficulty        int64
baseFeePerGas     int64
gasLimit          int64
gasUsed           int64
hash             object
miner            object
size              int64
nonce            object
dtype: object

In [7]:
block_data.head()

,number,timestamp,difficulty,baseFeePerGas,gasLimit,gasUsed,hash,miner,size,nonce
0,13438609,1634516292,9722652340559899,59461717512,30000000,29999522,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,0x00192Fb10dF37c9FB26829eb2CC623cd1BF599E8,128861,0xe5f34430a9014d68


In [48]:
# Basic cleansing
def transform(df):
    df.columns = df.columns.str.strip().str.replace(' ', '_').str.upper().str.replace('(','')
    objs = df.select_dtypes(['object'])
    df[objs.columns] = objs.apply(lambda x: x.str.strip())
    
    return df

In [51]:
transform(block_data)
transform(trx_data)

/tmp/ipykernel_14045/1038297608.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.replace(' ', '_').str.upper().str.replace('(','')


,ACCESSLIST,BLOCKHASH,BLOCKNUMBER,CHAINID,FROM,GAS,GASPRICE,HASH,INPUT,MAXFEEPERGAS,MAXPRIORITYFEEPERGAS,NONCE,R,S,TO,TRANSACTIONINDEX,TYPE,V,VALUE
0,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0x14da174FaB1356466aF2dD0ecdF94204E828A732,183152,61461717512,0x12c85149be06f6be5b4923cd5146a0b928c176bdc864...,0x38ed1739000000000000000000000000000000000000...,9.386950e+10,2.000000e+09,69,0xc52b779454d1d53f58dd612740747bea39a468c245f6...,0x45560e2f9ef8ae7c31873d3f24b825b7c7130bb542b3...,0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F,0,0x2,1,0
1,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0x59dDe211879e832B0E173C49f43574909598683b,22000,26258475007475,0xb7a1bebf863480d827cb84d3722f4de2b0b429213568...,0x,5.251695e+13,2.619901e+13,602,0x72c1f1209dfd81539ca7d2c1202fb3dce0160a7010fe...,0x0ad7ffeef580994d921e0e26ed71421e37953f4683b9...,0x59dDe211879e832B0E173C49f43574909598683b,1,0x2,1,62
2,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0x34b5f399cc5A1dD491666c9866941FB8E8D09746,261922,61461717512,0x067469399c9de4792dca4e709422e89e615c37e2fbdf...,0x0a661b1f000000000000000000000000000000000000...,1.258605e+11,2.000000e+09,1899,0x2380efea25398346330bc862f8993940f2ec945d3a0a...,0x0ffb19291b56fed504d3fa2814be9a39248be4272faf...,0xac397b34d4415fbfB9e445C371cc2D457D7f06c1,2,0x2,1,0
3,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,NaN,0x05f2fe99EA69ecAfd6ad2cDDc02F154A97413B5d,21000,92000000000,0x8f43c5a5cd1ad2f23495b9f39d494d9a4969c2e4bb7a...,0x,NaN,NaN,1447,0xb92be95062d94ac7cb62b1198eca9a71cc18a6a550e9...,0x25015a8abd090d31a2efbb933ba2adb27ca188d6ad6f...,0x965e8662151356Ec8048Bb801E91516AD9DC31c4,3,0x0,38,4000000000000000000
4,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,NaN,0x8FAa7dE59e0d0cD0f348fe06aCaAd5D1A539b6d8,200000,110400000000,0xe6693b0af7f0ca5bbb8bfd1f1eba2cd45c6ade58dc0a...,0xa9059cbb000000000000000000000000e5b8ff1ca1c3...,NaN,NaN,85,0x03b1d867bb84d1bf9c66d48a9623b13409dce8d166aa...,0x6e91f56557dd72b900f2296c40077c9478d4d555f895...,0x92D6C1e31e14520e676a687F0a93788B716BEff5,4,0x0,38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0x0BCa969176024bAb26890f520a74Edbc252734Ed,72912,60961717512,0xdf976b5255bb3b472ee56306eeda1d8ae9d4e9983572...,0x095ea7b3000000000000000000000000d9e1ce17f264...,7.762482e+10,1.500000e+09,1,0xd29e60b7549d9d7f03a811c3e49e08e5ae37012a37aa...,0x79969a1eef4fd8e24e42b29650a3a8d7022ae28cdeb8...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,303,0x2,0,0
304,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0x4Cb9fF3e7b3D461dE2013537F8Dd55671247128b,54022,60961717512,0x90557a68810648f101e9699309f6321e58515fd8a1fd...,0x2e1a7d4d000000000000000000000000000000000000...,7.762482e+10,1.500000e+09,18,0xe8f67a2adfe97dc528ef5e374e1c32845ecee02391bd...,0x79fa4b842b7937b424c205f1116c1f35b5e8821c2240...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,304,0x2,1,0
305,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0xd7553Af4503743F09b5d1FBd664a527F3aD0e5CA,21000,60961717512,0x5c83305f4722d7da4faf7b3287a24a510264081353a5...,0x,7.762482e+10,1.500000e+09,0,0xcc62383c7264438d6b3bdc8aa9e3ae1e9d6afce84ee3...,0x1b172f9453912b062a488ed30e44e80b9b335907be0c...,0xFb75B231C307738ce506c242bABaD2FD2e77B0bf,305,0x2,0,9994869878756438000
306,NaN,0xfec46debc7a913909a4043280f7ed09640514c7b6e56...,13438609,0x1,0xBB669550Bf85775785C42A1369816e0EA9b2A276,21000,60961717512,0x912a7fbf3051b6a79ebc9dddcefcd283a602449d89cb...,0x,7.762482e+10,1.500000e+09,13,0xfc0c978fa6a9461d6998bf3d6ee8eb61b01509142975...,0x42b784c1ad92b3c8fb57ad5ccf8a9b2d2d869afcb713...,0x95e93d907F44d553733b789e61321E40c350dF9c,306,0x2,0,9369567072054286


In [54]:
block_data.dtypes

NUMBER            int64
TIMESTAMP         int64
DIFFICULTY        int64
BASEFEEPERGAS     int64
GASLIMIT          int64
GASUSED           int64
HASH             object
MINER            object
SIZE              int64
NONCE            object
dtype: object

In [81]:
trx_data.dtypes

ACCESSLIST              float64
BLOCKHASH                object
BLOCKNUMBER               int64
CHAINID                  object
FROM                     object
GAS                       int64
GASPRICE                  int64
HASH                     object
INPUT                    object
MAXFEEPERGAS            float64
MAXPRIORITYFEEPERGAS    float64
NONCE                     int64
R                        object
S                        object
TO                       object
TRANSACTIONINDEX          int64
TYPE                     object
V                         int64
VALUE                    uint64
dtype: object

In [88]:
trx_data[['VALUE', 'BLOCKNUMBER']]

,VALUE,BLOCKNUMBER
0,0,13438609
1,62,13438609
2,0,13438609
3,4000000000000000000,13438609
4,0,13438609
...,...,...
303,0,13438609
304,0,13438609
305,9994869878756438000,13438609
306,9369567072054286,13438609


In [89]:
sql_manager = reload(sql_manager)
sql_manager.sqlcol(trx_data)

{'BLOCKHASH': VARCHAR(length=66),
 'BLOCKNUMBER': INTEGER(),
 'CHAINID': VARCHAR(length=3),
 'FROM': VARCHAR(length=42),
 'GAS': INTEGER(),
 'GASPRICE': INTEGER(),
 'HASH': VARCHAR(length=66),
 'INPUT': VARCHAR(length=5578),
 'MAXFEEPERGAS': VARCHAR(length=100),
 'MAXPRIORITYFEEPERGAS': VARCHAR(length=100),
 'NONCE': INTEGER(),
 'R': VARCHAR(length=66),
 'S': VARCHAR(length=66),
 'TO': VARCHAR(length=42),
 'TRANSACTIONINDEX': INTEGER(),
 'TYPE': VARCHAR(length=3),
 'V': INTEGER()}

In [58]:
trx_data.dtypes

ACCESSLIST              float64
BLOCKHASH                object
BLOCKNUMBER               int64
CHAINID                  object
FROM                     object
GAS                       int64
GASPRICE                  int64
HASH                     object
INPUT                    object
MAXFEEPERGAS            float64
MAXPRIORITYFEEPERGAS    float64
NONCE                     int64
R                        object
S                        object
TO                       object
TRANSACTIONINDEX          int64
TYPE                     object
V                         int64
VALUE                    uint64
dtype: object

In [9]:
# convert hash into readable hex format
trx_data.r = trx_data.r.apply(lambda x: web3.toHex(x))

trx_data.s = trx_data.s.apply(lambda x: web3.toHex(x))

trx_data.hash = trx_data.hash.apply(lambda x: web3.toHex(x))

trx_data.blockHash = trx_data.blockHash.apply(lambda x: web3.toHex(x))

In [37]:
# Define filenames
block_data_filename = f'eth_blockmetadata_{number}_{block.timestamp}'
trx_data_filename = f'eth_blocktransactions_{number}_{block.timestamp}'

In [90]:
trx_data.dtypes

BLOCKHASH                object
BLOCKNUMBER               int64
CHAINID                  object
FROM                     object
GAS                       int64
GASPRICE                  int64
HASH                     object
INPUT                    object
MAXFEEPERGAS            float64
MAXPRIORITYFEEPERGAS    float64
NONCE                     int64
R                        object
S                        object
TO                       object
TRANSACTIONINDEX          int64
TYPE                     object
V                         int64
VALUE                    uint64
dtype: object

In [53]:
# Local Data dumps
trx_data.to_csv(os.path.join('data',f'{trx_data_filename}.csv'), index=False, sep = ',', encoding='utf-8')
trx_data.to_json(os.path.join('data',f'{trx_data_filename}.json'), orient='records', default_handler=str)

In [52]:
block_data.to_csv(os.path.join('data',f'{block_data_filename}.csv'), index=False, sep = ',', encoding='utf-8')
block_data.to_json(os.path.join('data',f'{block_data_filename}.json'), orient='records', default_handler=str)

In [35]:

# S3 Data dumps
S3_BUCKET = 'blockchainsp'

s3_services.upload(os.path.join('data',f'{trx_data_filename}.csv'), S3_BUCKET, f'{trx_data_filename}.csv')
s3_services.upload(os.path.join('data',f'{trx_data_filename}.csv'), S3_BUCKET, f'{trx_data_filename}.json')
s3_services.upload(os.path.join('data',f'{block_data_filename}.csv'), S3_BUCKET, f'{block_data_filename}.csv')
s3_services.upload(os.path.join('data',f'{block_data_filename}.csv'), S3_BUCKET, f'{block_data_filename}.json')